<h1>Spotify API Listening Report</h1>

<h4>Sophia Swengel</h4>

October 15, 2023

<hr/>

<p>Discussion of Spotify's algorithms can often seem more common that discussion of the app itself, with users discovering and exploring new music through Spotify's automated recommendation system and playlists every day. I personally find its algorithms fascinating and utilize them in my use of the app. As a listener of many types of music often disparate in sound, genre, and popularity, I seek to ask <b>which is more important to Spotify's recommendation algorithm: song characteristics (or 'audio features') or song popularity?</b></p>

<p>In my personal experience, recommendations are highly tied to popularity. When I listen to artists who have pushed millions, even when listening to more obscure or unorthodox sounding works by them, I get artists who push millions coming up after them regardless of sonics. When I listen to obscure independent groups, however, I always get bands I've never heard of with a similar sound coming up next. From this, I hypothesize that <b>Spotify's recommendation system values song popularity over audio features.</b></p>

<p>Our hypothesis will be tested with the song "Astronomy Domine" by Pink Floyd for numerous reasons:</p>

<ol>
    <li>It is a great song.</li>
    <li>Pink Floyd are a very well known and widely listened to band.</li>
    <li>This song in particular straddles an interesting line in the band's discography. It is a beloved and well recognizable track, but it is closer sonically to garage rock music of the time, whose artists are often very unpopular compared to Pink Floyd, than the grandiose sounds that would grant the band international fame later in their career. This <i>does</i> open up the possibility of more obscure sounds appearing in its recommendations through shared audio features.</li>
</ol>

<h3>Process</h3>

<p>We can use the Spotify API to examine the recommendations of a song on Spotify, their audio features, and their values of popularity.</p>

<p>First, we import all of the assets we will need to analyze the data as well as the client ID, secret, and credibility needed to access the Spotify API.</p>

In [1]:
import requests
import pandas as pd
import base64
import json
import urllib

In [2]:
client_id = pd.read_csv('Spotify_Keys_September_19.txt')['Client_ID'].iloc[0]
# client_id

In [3]:
client_secret = pd.read_csv('Spotify_Keys_September_19.txt')['Client_Secret'].iloc[0]
# client_secret

In [4]:
client_cred = base64.b64encode(str(client_id + ":" + client_secret).encode('ascii'))
# client_cred

Then, we load the API headers and session headers.

In [5]:
headers = {"Authorization": "Basic {}".format(client_cred.decode("ascii"))}
# headers

In [6]:
payload = {'grant_type' : 'client_credentials'}
url = 'https://accounts.spotify.com/api/token'

In [7]:
session_key_response = requests.post(url = url, data = payload, headers = headers)
session_key_response.status_code

200

In [8]:
session_header_key = session_key_response.json()

In [9]:
key = session_header_key['access_token']

In [10]:
session_headers = {"Authorization": "Bearer {}".format(key)}
# session_headers

<h4>The song itself</h4>

<p>Going into the Spotify application allows us to view the ID of a song. We use this ID to load our chosen song into our data analysis with the API.</p>

In [11]:
pf_id = '3Y8gNcoJu1uJ5L1BZuGvqB'

In [12]:
track_url = 'https://api.spotify.com/v1/tracks/{}'.format(pf_id)

In [13]:
pf_response = requests.get(url = track_url, headers = session_headers)
pf_response

<Response [200]>

Audio features of the song itself

In [14]:
pf_features_url = 'https://api.spotify.com/v1/audio-features/{}'.format(pf_id)
pf_features_url

'https://api.spotify.com/v1/audio-features/3Y8gNcoJu1uJ5L1BZuGvqB'

In [15]:
pf_features_response = requests.get(url = pf_features_url, headers = session_headers)
pf_features_response.status_code

200

In [16]:
pf_features_data = pf_features_response.json()
pf_features_data

{'danceability': 0.348,
 'energy': 0.633,
 'key': 2,
 'loudness': -11.527,
 'mode': 1,
 'speechiness': 0.048,
 'acousticness': 0.376,
 'instrumentalness': 0.473,
 'liveness': 0.346,
 'valence': 0.0805,
 'tempo': 124.096,
 'type': 'audio_features',
 'id': '3Y8gNcoJu1uJ5L1BZuGvqB',
 'uri': 'spotify:track:3Y8gNcoJu1uJ5L1BZuGvqB',
 'track_href': 'https://api.spotify.com/v1/tracks/3Y8gNcoJu1uJ5L1BZuGvqB',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3Y8gNcoJu1uJ5L1BZuGvqB',
 'duration_ms': 251279,
 'time_signature': 4}

<h4>Song recommendations</h4>

<p>From there we can use the API to access the song's recommendations.</p>

In [17]:
pf_rec_url = 'https://api.spotify.com/v1/recommendations?seed_tracks={}'.format(pf_id)
pf_rec_url

'https://api.spotify.com/v1/recommendations?seed_tracks=3Y8gNcoJu1uJ5L1BZuGvqB'

In [18]:
pf_rec_response = requests.get(url = pf_rec_url, headers = session_headers)
pf_rec_response.status_code

200

In [19]:
pf_rec_data = pf_rec_response.json()
#pf_rec_data

<p>The data sheet of the recommendation data is very long due to the number of songs and, most infuriatingly, individual listings for every region a song is available in. Using .keys() we can navigate through the data to access the data relevant to our experiment.</p>

In [20]:
pf_rec_data.keys()

dict_keys(['tracks', 'seeds'])

<p>We're looking for the popularity of each track, so we navigate to the data properties for the tracks.</p>

In [21]:
pf_rec_data['tracks'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [22]:
pf_rec_data['tracks'][0]['popularity']

39

<p>We can also navigate to the name of the artist that created the song, which will come in handy when assembling the final dataframe in terms of song identification.</p>

In [23]:
pf_rec_data['tracks'][0]['artists'][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [24]:
pf_rec_data['tracks'][0]['artists'][0]['name']

'The Yardbirds'

<p>From there we can make a dataframe that displays the track properties, including that of popularity.</p>

In [25]:
pf_rec_df = pd.DataFrame(pf_rec_data['tracks'])
pf_rec_df.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,166213,False,{'isrc': 'GBAWA0175420'},{'spotify': 'https://open.spotify.com/track/48...,https://api.spotify.com/v1/tracks/48fcwp3ZkLYl...,48fcwp3ZkLYlhk8xUL7Rib,False,"Stroll On (From ""Blow Up"") - 2015 Remaster",39,https://p.scdn.co/mp3-preview/25db4279c40b4528...,9,track,spotify:track:48fcwp3ZkLYlhk8xUL7Rib
1,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,242640,False,{'isrc': 'USSM17100736'},{'spotify': 'https://open.spotify.com/track/0M...,https://api.spotify.com/v1/tracks/0M7oqERflkrJ...,0M7oqERflkrJVHD3IAZqxo,False,Cities On Flame with Rock and Roll,56,https://p.scdn.co/mp3-preview/0809252e43b3a86d...,8,track,spotify:track:0M7oqERflkrJVHD3IAZqxo
2,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,166453,False,{'isrc': 'GBA076700030'},{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0GxV3poTM7Z7...,0GxV3poTM7Z7fKSUqy5qFw,False,Tales Of Brave Ulysses,57,None,6,track,spotify:track:0GxV3poTM7Z7fKSUqy5qFw
3,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,172706,False,{'isrc': 'GB5KW1801357'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/43lZ8nY0TNIy...,43lZ8nY0TNIy2CX3BYCatV,False,Big Sky - 2018 Stereo Remaster,35,https://p.scdn.co/mp3-preview/37d9d0ca9aa7b3e4...,6,track,spotify:track:43lZ8nY0TNIy2CX3BYCatV
4,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AL, AM, AT, AZ, BA, BE, BG, BY, CH, CW, C...",1,273693,False,{'isrc': 'GBN9Y1100072'},{'spotify': 'https://open.spotify.com/track/4u...,https://api.spotify.com/v1/tracks/4uxrLzUlWG71...,4uxrLzUlWG71UR4jN1G4cL,False,Childhood's End - 2011 Remastered Version,42,https://p.scdn.co/mp3-preview/8a682f93eacd9102...,7,track,spotify:track:4uxrLzUlWG71UR4jN1G4cL


<p>This dataframe will be merged with the dataframe for audio properties later.</p>

<p>In order to call the audio properties for the recommendations, we can convert the 'id' column into a list of the IDs of the recommended songs and then insert it into the Spotify API.</p>

In [26]:
rec_list = ','.join(list(pf_rec_df['id']))

In [27]:
rec_features_url = 'https://api.spotify.com/v1/audio-features?ids={}'.format(rec_list)
rec_features_url

'https://api.spotify.com/v1/audio-features?ids=48fcwp3ZkLYlhk8xUL7Rib,0M7oqERflkrJVHD3IAZqxo,0GxV3poTM7Z7fKSUqy5qFw,43lZ8nY0TNIy2CX3BYCatV,4uxrLzUlWG71UR4jN1G4cL,7ua8ZMH06UdC3jcYPsEFPc,05f8Hg3RSfiPSCBQOtxl3i,3EmFqjkCftCBM7uKCql2Yn,6vSyKfgLccRMuTlwWI6Bi1,2IVyQNYOJSRgkSgSH8y5cb,6UrgPc2f78pgDrdodMJA42,6ypqzijMjsopQkfMLrImQp,3FckgkjljxOUBt9LYUZo5Z,59ENhfT8TIFjB30TUVG4PA,0myRKEG1A20Ep57FDWzRQA,2NTH6pt0kRQc7wuDuFPzKO,042v1NacbKJzCyi6nBme7T,33yz4pZitBJaa7eXSW5Dxs,5lmGgOaixbwKHtpTzjJoqx,1gmO2YyFdY4Y3NEw3HlS9a'

In [28]:
rec_feat_response = requests.get(url = rec_features_url, headers = session_headers)
rec_feat_response.status_code

200

<p>We can view the data of the audio features by converting the data to .json and using .keys() to navigate the data.</p>

In [29]:
rec_feat_data = rec_feat_response.json()
rec_feat_data.keys()

dict_keys(['audio_features'])

In [30]:
rec_feat_data['audio_features'][0]

{'danceability': 0.395,
 'energy': 0.897,
 'key': 11,
 'loudness': -8.778,
 'mode': 0,
 'speechiness': 0.11,
 'acousticness': 0.056,
 'instrumentalness': 0.000657,
 'liveness': 0.109,
 'valence': 0.307,
 'tempo': 91.226,
 'type': 'audio_features',
 'id': '48fcwp3ZkLYlhk8xUL7Rib',
 'uri': 'spotify:track:48fcwp3ZkLYlhk8xUL7Rib',
 'track_href': 'https://api.spotify.com/v1/tracks/48fcwp3ZkLYlhk8xUL7Rib',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/48fcwp3ZkLYlhk8xUL7Rib',
 'duration_ms': 166213,
 'time_signature': 4}

<p>Then we can create a dataframe of the audio features of each recommended song.</p>

In [31]:
rec_feat_df = pd.DataFrame(rec_feat_data['audio_features'])
rec_feat_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.395,0.897,11,-8.778,0,0.1100,0.05600,0.000657,0.1090,0.307,91.226,audio_features,48fcwp3ZkLYlhk8xUL7Rib,spotify:track:48fcwp3ZkLYlhk8xUL7Rib,https://api.spotify.com/v1/tracks/48fcwp3ZkLYl...,https://api.spotify.com/v1/audio-analysis/48fc...,166213,4
1,0.337,0.886,4,-8.607,1,0.0980,0.07160,0.001560,0.2040,0.648,177.713,audio_features,0M7oqERflkrJVHD3IAZqxo,spotify:track:0M7oqERflkrJVHD3IAZqxo,https://api.spotify.com/v1/tracks/0M7oqERflkrJ...,https://api.spotify.com/v1/audio-analysis/0M7o...,242640,4
2,0.538,0.476,0,-9.215,0,0.0490,0.01780,0.000043,0.1010,0.188,105.642,audio_features,0GxV3poTM7Z7fKSUqy5qFw,spotify:track:0GxV3poTM7Z7fKSUqy5qFw,https://api.spotify.com/v1/tracks/0GxV3poTM7Z7...,https://api.spotify.com/v1/audio-analysis/0GxV...,166453,4
3,0.553,0.798,10,-7.405,1,0.0274,0.00659,0.000612,0.0923,0.560,98.285,audio_features,43lZ8nY0TNIy2CX3BYCatV,spotify:track:43lZ8nY0TNIy2CX3BYCatV,https://api.spotify.com/v1/tracks/43lZ8nY0TNIy...,https://api.spotify.com/v1/audio-analysis/43lZ...,172707,4
4,0.672,0.321,9,-16.789,1,0.0435,0.13600,0.000046,0.0934,0.202,125.041,audio_features,4uxrLzUlWG71UR4jN1G4cL,spotify:track:4uxrLzUlWG71UR4jN1G4cL,https://api.spotify.com/v1/tracks/4uxrLzUlWG71...,https://api.spotify.com/v1/audio-analysis/4uxr...,273693,4


<p>Then, we can merge the dataframe of the <b>recommendation properties</b> (including <b>popularity</b>) with the dataframe of <b>the audio features of the recommendations</b>.</p>

In [32]:
id_frame = pd.merge(pf_rec_df, rec_feat_df, how = 'inner', on = 'id')
id_frame.head()

,album,artists,available_markets,disc_number,duration_ms_x,explicit,external_ids,external_urls,href,id,...,instrumentalness,liveness,valence,tempo,type_y,uri_y,track_href,analysis_url,duration_ms_y,time_signature
0,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,166213,False,{'isrc': 'GBAWA0175420'},{'spotify': 'https://open.spotify.com/track/48...,https://api.spotify.com/v1/tracks/48fcwp3ZkLYl...,48fcwp3ZkLYlhk8xUL7Rib,...,0.000657,0.1090,0.307,91.226,audio_features,spotify:track:48fcwp3ZkLYlhk8xUL7Rib,https://api.spotify.com/v1/tracks/48fcwp3ZkLYl...,https://api.spotify.com/v1/audio-analysis/48fc...,166213,4
1,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,242640,False,{'isrc': 'USSM17100736'},{'spotify': 'https://open.spotify.com/track/0M...,https://api.spotify.com/v1/tracks/0M7oqERflkrJ...,0M7oqERflkrJVHD3IAZqxo,...,0.001560,0.2040,0.648,177.713,audio_features,spotify:track:0M7oqERflkrJVHD3IAZqxo,https://api.spotify.com/v1/tracks/0M7oqERflkrJ...,https://api.spotify.com/v1/audio-analysis/0M7o...,242640,4
2,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,166453,False,{'isrc': 'GBA076700030'},{'spotify': 'https://open.spotify.com/track/0G...,https://api.spotify.com/v1/tracks/0GxV3poTM7Z7...,0GxV3poTM7Z7fKSUqy5qFw,...,0.000043,0.1010,0.188,105.642,audio_features,spotify:track:0GxV3poTM7Z7fKSUqy5qFw,https://api.spotify.com/v1/tracks/0GxV3poTM7Z7...,https://api.spotify.com/v1/audio-analysis/0GxV...,166453,4
3,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,172706,False,{'isrc': 'GB5KW1801357'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/43lZ8nY0TNIy...,43lZ8nY0TNIy2CX3BYCatV,...,0.000612,0.0923,0.560,98.285,audio_features,spotify:track:43lZ8nY0TNIy2CX3BYCatV,https://api.spotify.com/v1/tracks/43lZ8nY0TNIy...,https://api.spotify.com/v1/audio-analysis/43lZ...,172707,4
4,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AL, AM, AT, AZ, BA, BE, BG, BY, CH, CW, C...",1,273693,False,{'isrc': 'GBN9Y1100072'},{'spotify': 'https://open.spotify.com/track/4u...,https://api.spotify.com/v1/tracks/4uxrLzUlWG71...,4uxrLzUlWG71UR4jN1G4cL,...,0.000046,0.0934,0.202,125.041,audio_features,spotify:track:4uxrLzUlWG71UR4jN1G4cL,https://api.spotify.com/v1/tracks/4uxrLzUlWG71...,https://api.spotify.com/v1/audio-analysis/4uxr...,273693,4


<p>We can sort this merged frame to better convey the data points we are interested in.</p>

<p>First, we can group the frame by the titles of songs to make the dataframe easy to digest. We can call the dataframe's columns and do away with those that are irrelevant to our project to maximize efficiency and ease of data digestion.</p>

In [33]:
id_frame.columns

Index(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms_x',
       'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local',
       'name', 'popularity', 'preview_url', 'track_number', 'type_x', 'uri_x',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_y', 'uri_y', 'track_href', 'analysis_url', 'duration_ms_y',
       'time_signature'],
      dtype='object')

In [34]:
id_frame = id_frame.drop(columns=['album', 'available_markets', 'disc_number', 'duration_ms_x', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'preview_url', 'track_number', 'type_x', 'uri_x', 'key', 'mode', 'type_y', 'uri_y', 'track_href', 'analysis_url', 'duration_ms_y', 'time_signature'])
id_frame.head(1)

,artists,name,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,[{'external_urls': {'spotify': 'https://open.s...,"Stroll On (From ""Blow Up"") - 2015 Remaster",39,0.395,0.897,-8.778,0.11,0.056,0.000657,0.109,0.307,91.226


<p>Previously we had navigated through some of our data to find data relevant for making our dataframe easy to digest. Let's use x for x list comprehension to make lists and then columns of our frame out of this data.</p>

In [35]:
id_frame['artists'] = artist_list = [x['artists'][0]['name'] for x in pf_rec_data['tracks']]
id_frame['popularity'] = pop_list = [x['popularity'] for x in pf_rec_data['tracks']]

<p>We can do the same for the audio features data we took a look at earlier. Not all of the audio features are included, such as key and mode, are not included because they don't reflect the musical qualities that the average listener is likely thinking about while listening to a song.</p>

In [36]:
id_frame['danceability'] = dance_list = [x['danceability'] for x in rec_feat_data['audio_features']]
id_frame['energy'] = energy_list = [x['energy'] for x in rec_feat_data['audio_features']]
id_frame['loudness'] = loudness_list = [x['loudness'] for x in rec_feat_data['audio_features']]
id_frame['speechiness'] = speechiness_list = [x['speechiness'] for x in rec_feat_data['audio_features']]
id_frame['acousticness'] = acousticness_list = [x['acousticness'] for x in rec_feat_data['audio_features']]
id_frame['instrumentalness'] = instrumentalness_list = [x['instrumentalness'] for x in rec_feat_data['audio_features']]
id_frame['liveness'] = liveness_list = [x['liveness'] for x in rec_feat_data['audio_features']]
id_frame['valence'] = valence_list = [x['valence'] for x in rec_feat_data['audio_features']]
id_frame['tempo'] = tempo_list = [x['tempo'] for x in rec_feat_data['audio_features']]

<p>Now that we have all of these columns lined up for our frame, let's generate it.</p>

In [37]:
id_frame

,artists,name,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,The Yardbirds,"Stroll On (From ""Blow Up"") - 2015 Remaster",39,0.395,0.897,-8.778,0.1100,0.056000,0.000657,0.1090,0.307,91.226
1,Blue Öyster Cult,Cities On Flame with Rock and Roll,56,0.337,0.886,-8.607,0.0980,0.071600,0.001560,0.2040,0.648,177.713
2,Cream,Tales Of Brave Ulysses,57,0.538,0.476,-9.215,0.0490,0.017800,0.000043,0.1010,0.188,105.642
3,The Kinks,Big Sky - 2018 Stereo Remaster,35,0.553,0.798,-7.405,0.0274,0.006590,0.000612,0.0923,0.560,98.285
4,Pink Floyd,Childhood's End - 2011 Remastered Version,42,0.672,0.321,-16.789,0.0435,0.136000,0.000046,0.0934,0.202,125.041
5,Funkadelic,"Mommy, What's A Funkadelic?",32,0.357,0.654,-11.957,0.0773,0.469000,0.001320,0.6270,0.641,146.176
6,Led Zeppelin,When the Levee Breaks - Remaster,65,0.271,0.766,-8.030,0.0329,0.002170,0.520000,0.0707,0.803,142.914
7,Strawberry Alarm Clock,Pretty Song From Psych-Out,27,0.513,0.460,-10.609,0.0292,0.247000,0.000000,0.1220,0.661,110.859
8,The Zodiac,Aries - The Fire Fighter,17,0.336,0.655,-9.521,0.0624,0.364000,0.370000,0.1140,0.446,89.632
9,Syd Barrett,Octopus,36,0.584,0.445,-11.226,0.0342,0.446000,0.000002,0.1300,0.510,88.161


<h3>Data analysis</h3>

<p>The Spotify API pages for <a href="https://developer.spotify.com/documentation/web-api/reference/get-recommendations">getting recommendations</a> and <a href="https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features">getting audio features</a> give definitions for what each of the values mean that can help us analyze our data table.</p>

<p>Every time the data is run, it generates a new set of recommendations. This data analysis is based on the screenshot of the dataframe below from the first ten songs.</p>

<img src="api_table.png" alt="API table" />

<h4>Popularity</h4>

<p>Popularity is measured on a scale of 0-100 with 100 being the most popular. The value is time relative and is mostly based on "the total number of plays the track has had and how recent those plays are".</p>

<ul>
    <li>The popularities of the recommended tracks range from 21 (quite unpopular) to 68, with a range of 47. Reduced to a scale of 0.0-1.0 like most of the audio features, The range would be 0.47.</li>
</ul>
  
<h4>Audio Features</h4>

<p>Acousticness, danceability, energy, instrumentalness, liveliness, speechiness, and valence (<a href="https://community.spotify.com/t5/Spotify-for-Developers/Valence-as-a-measure-of-happiness/td-p/4385221">"musical happiness"</a>) are measured on a scale of 0.0-1.0 with 1.0 being the the highest detected amount of the quality being measured. Loudness is measured in decibels and tempo is the estimated beats per minute.</p>

<p>To examine a few choice audio features for simplicity's sake,</p>

<ul>
    <li>the danceability range from 0.293 to 0.677, with a range of 0.384.</li>
    <li>the energy range from 0.176 to 0.918; counting out the outliers of 0.176 and 0.265, the range is 0.585.</li>
    <li>the speechiness range from 0.0270 to 0.2060, with a range of 0.179.</li>
    <li>the acousticness range from 0.0134 to 0.6790, with a range of 0.6656.</li>
    <li>the instrumentalness range from 0.0 to 0.483.</li>
    <li>the liveness range from 0.0593 to 0.6070; counting out the outlier of 0.6070, the range is 0.3187.</li>
    <li>the valence range from 0.208 to 0.833; counting out the outliers of 0.208 and 0.287, the range is 0.474.</li>
</ul>

<h4>Conclusion</h4>

<p>The recommended tracks had varying degrees of popularity and varying measures of certain audio attributes. Most audio attributes had a range between 0.30 and 0.50, with the range of popularities at the high end. Across the board, the individual features of tracks varied, making it difficult to draw any conclusion about how any one feature determines that a track will be recommended. There isn't much of a linear answer to the hypothesis other than that there are other, more complex factors at play in the Spotify algorithm when generating recommended tracks.</p>

<p>The tracks can all be contained under the umbrella of psychedelic rock, and most (except for a random Tame Impala song) are from the era of the late 1960s and early 1970s, which would be expected, despite the often drastic variations of "audio features" measurements between tracks. Hence, examining audio features is not a very good way of gauging the actual sonics of a song.</p>

<p>It could be interesting to see how many of these songs are on playlists together considering the genre similarity, especially the outlier from Tame Impala, a modern group (who I've never even listened to, so I'm not sure why Spotify is recommending them to me specifically!).

<p>The variation between certain audio features in relation to each other would also be an interesting area to explore. I am interested in that the tracks greatly range in instrumentalness, but are quite close in terms of speechiness, likely lending to often-somewhat-buried vocals in psychedelic music. This could be explore how Spotify detects different audio features based on music genre and sonics. Audio features may not predict sonics, but sonics determine audio features, so how does Spotify's detection of certain features vary from genre to genre?</p>